In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
data = pd.read_csv('input/amazon-reviews-sample.csv')

In [ ]:
data.head()

In [ ]:
data['reviewText'].head()

In [ ]:
data.shape

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer 
stem = PorterStemmer()

def clean_text(input_text):
    input_text = str(input_text)
    words = input_text.split() 
    noise_free_words = [lem.lemmatize(word, "v") for word in words if word not in stop_words] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

In [ ]:
type(data['reviewText'][0])

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
nltk.download()

In [ ]:
data['cleaned_review'] = data['reviewText'].apply(clean_text)

In [ ]:
data.head()

In [ ]:
def find_sentiment(text_input):
    score = 0 
    text_input = text_input.split()
    f = open('input/pos.txt', 'r')
    x = f.read()
    x = x.split()
    f1 = open('input/neg.txt', 'r')
    y = f1.read()
    y = y.split()
    for i in text_input:
        if i in x:
            score = score + 1
        if i in y:
            score = score - 1
    return int(score)

In [ ]:
data['sentiment'] = data['cleaned_review'].apply(find_sentiment)

In [ ]:
data.to_csv("output/data_with_senti.csv")
data['sentiment'].head()

In [ ]:
data['sentiment'].value_counts()

In [ ]:
from nltk import word_tokenize, pos_tag
corpus = set()
new_corpus = []

def filter_features(text):
    tokens = word_tokenize(text)
    tokens = pos_tag(tokens)
    tokens = [word for word, pos in tokens if pos in ['NN']]
    corpus.update(tokens)
    new_corpus.extend(tokens)
    res = " ".join(tokens)
    return res

data['feature_text'] = data['cleaned_review'].apply(filter_features)
print(list(corpus)[:10])

In [ ]:
print(len(list(corpus)))
data['feature_text'].head()

In [ ]:
from collections import Counter
features_cnt = Counter(new_corpus)
common_features = features_cnt.most_common(50)
print(common_features)

In [ ]:
sents_bool = np.array(data['sentiment']) >= 0
print(len(sents_bool))
print(sents_bool[:10])

In [ ]:
sents_bool.mean()

In [ ]:
common_features_ = [x for x,y in common_features]
feat_dict = {}
sents = data['sentiment']
for idx, f in enumerate(common_features_):
    lst2 = [list(ft.split()).count(f) for ft in data['feature_text']]
    lst3 = []
    for x,b in zip(lst2, sents_bool):
        if b:
            lst3.append(x)
        else:
            lst3.append(-x)
    feat_dict[f] = lst3

In [ ]:
feat_dict['asin'] = data['asin']
feat_dict['title'] = data['title']

In [ ]:
pd.DataFrame(feat_dict)

In [ ]:
new_data = pd.DataFrame(feat_dict).iloc[:,:]
new_data = new_data.sort_values('title', ascending=False)
print(new_data.head())

In [ ]:
new_data2 = new_data.groupby('asin').sum()
print(new_data2.head())
#new_data = new_data.drop_duplicates(subset='asin', keep="last")

In [ ]:
#comment
from sklearn.feature_extraction.text import CountVectorizer
review_vectorizer = CountVectorizer()
review_features   = review_vectorizer.fit_transform(data['feature_text'])
review_features.get_shape()

In [ ]:
#comment
print(review_features[1000])
print(type(review_features))

In [ ]:
new_data2.shape

In [ ]:
new_data2.head(5)

In [ ]:
new_data2.to_csv("output/final_features_data.csv")

In [ ]:
new_data2.columns